In [1]:
import sys
sys.path.append('..')

In [7]:
import pandas as pd
import numpy as np

In [13]:
df = pd.read_csv('../csv/df_company_info.csv')
df

,stock_code,company_name,short_name,en_company_name,en_short_name,combine_profile,industry,exchange,foreign_percent,issue_share,no_shareholders,stock_rating,website
0,BID,Ngân hàng Thương mại Cổ phần Đầu tư và Phát tr...,BIDV,Joint Stock Commercial Bank for Investment and...,BIDV,company_name: Ngân hàng Thương mại Cổ phần Đầu...,Banking,HOSE,0.171,5.700400e+09,27603.0,2.8,https://www.bidv.com.vn
1,EIB,Ngân hàng Thương mại Cổ phần Xuất nhập khẩu Vi...,Eximbank,Vietnam Export Import Commercial Joint Stock Bank,Eximbank,company_name: Ngân hàng Thương mại Cổ phần Xuấ...,Banking,HOSE,0.028,1.868800e+09,25371.0,3.0,https://eximbank.com.vn
2,OCB,Ngân hàng Thương mại Cổ phần Phương Đông,Ngân hàng Phương Đông,Orient Commercial Joint Stock Bank,OCB Bank,company_name: Ngân hàng Thương mại Cổ phần Phư...,Banking,HOSE,0.200,2.465800e+09,20103.0,2.5,https://www.ocb.com.vn
3,CTG,Ngân hàng Thương mại Cổ phần Công thương Việt Nam,VietinBank,Vietnam Joint Stock Commercial Bank For Indust...,VietinBank,company_name: Ngân hàng Thương mại Cổ phần Côn...,Banking,HOSE,0.265,5.370000e+09,53928.0,3.0,http://www.vietinbank.vn
4,VCB,Ngân hàng Thương mại Cổ phần Ngoại thương Việt...,Vietcombank,Joint Stock Commercial Bank For Foreign Trade ...,Vietcombank,company_name: Ngân hàng Thương mại Cổ phần Ngo...,Banking,HOSE,0.233,5.589100e+09,25183.0,2.9,https://vietcombank.com.vn
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,ORS,Công ty Cổ phần Chứng khoán Tiên Phong,Chứng khoán Tiên Phong,Tien Phong Securities Corporation,Tien Phong Securities,company_name: Công ty Cổ phần Chứng khoán Tiên...,Financial Services,HOSE,0.015,3.360000e+08,12142.0,3.1,https://tpbs.com.vn
80,SCIC,Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước...,Đầu tư nhà nước,State Capital Investment and Trading Corporati...,State Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.scic.vn/index.php/vi/
81,Viettel,Tập đoàn Công nghiệp - Viễn thông Quân đội,Viettel,Viettel Group,Viettel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://viettel.vn
82,NHNN,Ngân hàng Nhà nước Việt Nam,NHNN,State Bank of Vietnam,Central Bank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://sbv.gov.vn


In [19]:
df_profile = df[['stock_code','en_short_name', 'en_company_name', 'industry', 'exchange']]

In [24]:
df_sub = pd.read_csv('../csv/df_sub_and_shareholders.csv')

In [32]:


# Step 1: Identify potential mismatches
# Create a mask for rows where invest_on and stock_code are swapped
mask = df_sub['invest_on'].isin(df_sub['stock_code']) & df_sub['stock_code'].isin(df_sub['invest_on'])

# Step 2: Create a DataFrame for swapped rows
swapped_rows = df_sub[mask]

# Step 3: Iterate through swapped rows and keep the one with higher percentage
to_keep = []

for index, row in swapped_rows.iterrows():
    # Get the corresponding row with swapped values
    corresponding_row = df_sub[(df_sub['invest_on'] == row['stock_code']) & (df_sub['stock_code'] == row['invest_on'])]
    
    if not corresponding_row.empty:
        # print(row)
        # print(corresponding_row)
        # Compare percentages
        if row['percent'] > corresponding_row['percent'].values[0]:
            pass
        else:
            # to_keep.append(corresponding_row.iloc[0])
            continue

    to_keep.append(row)

df_final = pd.concat([df_sub[~mask], pd.DataFrame(to_keep)], ignore_index=True)
print(df_final)
df_final

     percent invest_on stock_code
0     0.8099       BID       NHNN
1     0.6446       CTG       NHNN
2     0.7480       VCB       NHNN
3     0.1476       MBB    Viettel
4     0.0986       MBB       SCIC
..       ...       ...        ...
351   0.0440       MBB        VCB
352   0.0000       OCB        VCB
353   0.0110       VJC        HDB
354   0.0000       CTG        MBS
355   0.0000       CTG        ORS

[356 rows x 3 columns]


,percent,invest_on,stock_code
0,0.8099,BID,NHNN
1,0.6446,CTG,NHNN
2,0.7480,VCB,NHNN
3,0.1476,MBB,Viettel
4,0.0986,MBB,SCIC
...,...,...,...
351,0.0440,MBB,VCB
352,0.0000,OCB,VCB
353,0.0110,VJC,HDB
354,0.0000,CTG,MBS


In [33]:
df_final.to_csv('../csv/df_sub_and_shareholders.csv', index=False)

In [36]:
df_final = df_final.merge(df_profile[['stock_code']], left_on='invest_on', right_on='stock_code', how='inner')
df_final

,percent,invest_on,stock_code_x,stock_code_y
0,0.8099,BID,NHNN,BID
1,0.6446,CTG,NHNN,CTG
2,0.7480,VCB,NHNN,VCB
3,0.1476,MBB,Viettel,MBB
4,0.0986,MBB,SCIC,MBB
5,0.1488,TCB,MSN,TCB
6,0.3600,VNM,SCIC,VNM
7,0.0575,FPT,SCIC,FPT
8,0.9576,GAS,PVN,GAS
9,0.1837,VRE,VIC,VRE


In [42]:
df_profile['Has Subsidiaries/ Invest on other company'] = df_profile['stock_code'].apply(lambda x: 'Yes' if x in df_sub['stock_code_x'].values else 'No')
df_profile

C:\Users\Admin\AppData\Local\Temp\ipykernel_24176\702412969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_profile['Has Subsidiaries/ Invest on other company'] = df_profile['stock_code'].apply(lambda x: 'Yes' if x in df_sub['stock_code_x'].values else 'No')


,stock_code,en_short_name,en_company_name,industry,exchange,Has Subsidiaries,Has Subsidiaries/ Invest on other company
0,BID,BIDV,Joint Stock Commercial Bank for Investment and...,Banking,HOSE,Yes,Yes
1,EIB,Eximbank,Vietnam Export Import Commercial Joint Stock Bank,Banking,HOSE,Yes,Yes
2,OCB,OCB Bank,Orient Commercial Joint Stock Bank,Banking,HOSE,Yes,Yes
3,CTG,VietinBank,Vietnam Joint Stock Commercial Bank For Indust...,Banking,HOSE,No,No
4,VCB,Vietcombank,Joint Stock Commercial Bank For Foreign Trade ...,Banking,HOSE,Yes,Yes
...,...,...,...,...,...,...,...
79,ORS,Tien Phong Securities,Tien Phong Securities Corporation,Financial Services,HOSE,Yes,Yes
80,SCIC,State Fund,State Capital Investment and Trading Corporati...,NaN,NaN,Yes,Yes
81,Viettel,Viettel,Viettel Group,NaN,NaN,Yes,Yes
82,NHNN,Central Bank,State Bank of Vietnam,NaN,NaN,Yes,Yes


In [43]:
company_table = df_profile.to_markdown()
print(company_table)

|    | stock_code   | en_short_name                           | en_company_name                                                              | industry                             | exchange   | Has Subsidiaries   | Has Subsidiaries/ Invest on other company   |
|---:|:-------------|:----------------------------------------|:-----------------------------------------------------------------------------|:-------------------------------------|:-----------|:-------------------|:--------------------------------------------|
|  0 | BID          | BIDV                                    | Joint Stock Commercial Bank for Investment and Development of Vietnam        | Banking                              | HOSE       | Yes                | Yes                                         |
|  1 | EIB          | Eximbank                                | Vietnam Export Import Commercial Joint Stock Bank                            | Banking                              | HOSE       | Yes                

In [48]:
from llm.llm.gemini import Gemini
from llm.llm_utils import get_json_from_text_response

In [45]:
main_tasks = [
    "Financial Ratios", "Accounts in Financial Statements", "Both Financial Ratios and Accounts in Financial Statements", "DuPont Analysis",
]

sub_tasks = [
    "get data 1 or more company", "compare 2 or more company", "analyze the Subsidiaries or invested company", "analyze over industry average report", "analyze company with its industry average report", "Ranking (Top 5 - Top 10)", "Ranking with special criterias (Top 5 - Top 10)(e.g: total asset >100B VND,  ROE > 20%)",
]

times = [
    "at specific year", "at specific quater and year", "over time with specific period",
]
len(main_tasks)* len(sub_tasks)* len(times)


84

In [50]:
def generate_questions(llm, main_task, sub_task, time, company_table):
    system_prompt = "You are a head of the investment fund and you are trying give tasks to your team to analyze the information from financial report of company from 2018 to Q2 2024."
    
    messages = [
        {
            'role': 'system',
            'content': system_prompt
        },
        {
            'role': 'user',
            'content': f"""Here are the details of the company
{company_table}
Notice that there are 4 company that was not listed on any exchange, since they are government company, and they only have data about ownership/shareholder of other companies. 
Generate 5-10 questions on {main_task} with {sub_task} {time}
Return the questions in a JSON format
{{
    "questions": []
}}

"""
        }
    ]
    response = llm(messages)
    
    output = {
        'main_task': main_task,
        'sub_task': sub_task,
        'time': time,
        'questions': []
    }
    
    try:
        output['questions'] = get_json_from_text_response(response, new_method=True)['questions']
    
    except Exception as e:
        print(e)
        
    return output
        
def parallel_generate_questions(main_task, sub_task, time):
    llm = Gemini()
    return generate_questions(llm, main_task, sub_task, time, company_table)
    

In [51]:
task = []
for main_task in main_tasks:
    for sub_task in sub_tasks:
        for time in times:
            task.append((main_task, sub_task, time))
            
task

[('Financial Ratios', 'get data 1 or more company', 'at specific year'),
 ('Financial Ratios',
  'get data 1 or more company',
  'at specific quater and year'),
 ('Financial Ratios',
  'get data 1 or more company',
  'over time with specific period'),
 ('Financial Ratios', 'compare 2 or more company', 'at specific year'),
 ('Financial Ratios',
  'compare 2 or more company',
  'at specific quater and year'),
 ('Financial Ratios',
  'compare 2 or more company',
  'over time with specific period'),
 ('Financial Ratios',
  'analyze the Subsidiaries or invested company',
  'at specific year'),
 ('Financial Ratios',
  'analyze the Subsidiaries or invested company',
  'at specific quater and year'),
 ('Financial Ratios',
  'analyze the Subsidiaries or invested company',
  'over time with specific period'),
 ('Financial Ratios',
  'analyze over industry average report',
  'at specific year'),
 ('Financial Ratios',
  'analyze over industry average report',
  'at specific quater and year'),
 ('F